## Importing the Dataset

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('../input/churn-predictions-personal/Churn_Predictions.csv')

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
tf.__version__

'2.4.1'

In [5]:
X = df.iloc[:, 3:-1].values
y = df.iloc[:, -1].values

In [6]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [7]:
print(y)

[1 0 1 ... 1 1 0]


## Encoding categorical data

### Label Encoding the "Gender" column

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

In [9]:
X

array([[619, 'France', 0, ..., 1, 1, 101348.88],
       [608, 'Spain', 0, ..., 0, 1, 112542.58],
       [502, 'France', 0, ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 0, ..., 0, 1, 42085.58],
       [772, 'Germany', 1, ..., 1, 0, 92888.52],
       [792, 'France', 0, ..., 1, 0, 38190.78]], dtype=object)

### One Hot Encoding the "Geography" column

In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [11]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


## Splitting the dataset into the Training and Test set

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8)

## Feature Scaling

In [13]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [14]:
X_train

array([[ 0.98930717, -0.57677292, -0.56965192, ...,  0.64356595,
         0.9770141 ,  1.25024145],
       [-1.01080841, -0.57677292,  1.75545796, ..., -1.55384232,
         0.9770141 , -0.9728827 ],
       [ 0.98930717, -0.57677292, -0.56965192, ..., -1.55384232,
         0.9770141 , -1.65070518],
       ...,
       [-1.01080841, -0.57677292,  1.75545796, ...,  0.64356595,
        -1.02352668,  0.55465689],
       [-1.01080841,  1.73378459, -0.56965192, ..., -1.55384232,
        -1.02352668, -0.73600498],
       [-1.01080841,  1.73378459, -0.56965192, ..., -1.55384232,
         0.9770141 ,  1.72775283]])

In [15]:
X_test

array([[-1.01080841,  1.73378459, -0.56965192, ...,  0.64356595,
         0.9770141 , -0.3415316 ],
       [-1.01080841,  1.73378459, -0.56965192, ...,  0.64356595,
         0.9770141 ,  0.3550058 ],
       [-1.01080841, -0.57677292,  1.75545796, ...,  0.64356595,
        -1.02352668,  1.22975581],
       ...,
       [-1.01080841,  1.73378459, -0.56965192, ...,  0.64356595,
        -1.02352668,  1.61377334],
       [ 0.98930717, -0.57677292, -0.56965192, ...,  0.64356595,
        -1.02352668,  0.56451317],
       [ 0.98930717, -0.57677292, -0.56965192, ...,  0.64356595,
        -1.02352668,  1.05043074]])

# Building the ANN

## Initializing the ANN

In [16]:
ann = tf.keras.models.Sequential()

## Adding the input layer and the first hidden layer

In [17]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

## Adding the second hidden layer

In [18]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

## Adding the Output Layer

In [19]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Training the ANN

## Compiling the ANN

In [20]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

## Training the ANN on the Training Set

In [21]:
ann.fit(X_train, y_train, batch_size=20, epochs=150)

Epoch 1/150
400/400 [==============================] - 1s 848us/step - loss: 0.7612 - accuracy: 0.5489
Epoch 2/150
400/400 [==============================] - 0s 815us/step - loss: 0.4703 - accuracy: 0.7954
Epoch 3/150
400/400 [==============================] - 0s 817us/step - loss: 0.4191 - accuracy: 0.8142
Epoch 4/150
400/400 [==============================] - 0s 882us/step - loss: 0.4146 - accuracy: 0.8219
Epoch 5/150
400/400 [==============================] - 0s 851us/step - loss: 0.4110 - accuracy: 0.8241
Epoch 6/150
400/400 [==============================] - 0s 844us/step - loss: 0.3801 - accuracy: 0.8419
Epoch 7/150
400/400 [==============================] - 0s 867us/step - loss: 0.3706 - accuracy: 0.8435
Epoch 8/150
400/400 [==============================] - 0s 887us/step - loss: 0.3635 - accuracy: 0.8483
Epoch 9/150
400/400 [==============================] - 0s 849us/step - loss: 0.3631 - accuracy: 0.8462
Epoch 10/150
400/400 [==============================] - 0s 857us/step - l

# Predicting for single customer

### Use our ANN model to predict if the customer with the following informations will leave the bank.
#### Geography: France
#### Credit Score: 600
#### Gender: Male
#### Age: 40 years old
#### Tenure: 3 years
#### Balance: 60000
#### Number of Products: 2
#### Does this customer have a credit card: Yes
#### Is this customer an Active Member: Yes
#### Estimated Salary: 50000
#### So, should we say goodbye to that customer ?

In [22]:
ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,5000]]))

array([[0.02944198]], dtype=float32)

In [23]:
ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,5000]])) > 0.5

array([[False]])

## Predicting the Test Result

In [24]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.3)
np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1)

array([[1, 0],
       [0, 1],
       [0, 0],
       ...,
       [0, 0],
       [0, 0],
       [1, 0]])

## Making the Confusion Matrix

In [25]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1429  158]
 [ 140  273]]


In [26]:
ac = accuracy_score(y_test, y_pred)

In [27]:
ac

0.851

In [28]:
threshold = [0.2,0.3,0.5,0.7,0.8]

for i in threshold:
    y_pred = ann.predict(X_test)
    y_pred = (y_pred > i)
    ac = accuracy_score(y_test, y_pred)
    print("when threshold is {} accuracy will be {}".format(i, ac*100))

when threshold is 0.2 accuracy will be 80.35
when threshold is 0.3 accuracy will be 85.1
when threshold is 0.5 accuracy will be 86.7
when threshold is 0.7 accuracy will be 85.2
when threshold is 0.8 accuracy will be 83.7
